# Info 

Build a matrix factorization-based recommendation model via singular valueu decomposition (SVD) and the `surprise` package. 

Mathematically, SVD "decomposes" a matrix into three other matrices. In practice for film ratings, the SVD model uses the training data of ratings (which becomes an N by M matrix of ratings for N users and M movies) to model a latent representations of individual users as well as individual films. These comprise 2 of the 3 resultant decomposition matrices, with the other being a matrix that defines the weighted importance of each r ating. 

The fitted model is then used to predict unseen ratings for a user based on these factors to serve as a recommendation system.

Steps:

1. Load and prepare ratings dataset (previously created via `prep_data_eda.ipynb`)
2. Train the SVD Model
3. Evaluate the Model
4. Make Predictions

Since the full dataset contains over 32 million rows of ratings, training here just for sandbox/exploratory syntax purposes is done with a random (25%) sample of the ratings, which should still provide a generous training set.

In [1]:
import os
import pandas as pd
import numpy as np

from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split, cross_validate
from surprise import accuracy

data_dir = '../data/'
df_ratings = pd.read_csv(os.path.join(data_dir, 'ratings_combined.csv'))

seed=77
sample_size=0.25
df_ratings_sample = df_ratings.sample(frac=sample_size, random_state=seed)

reader = Reader(rating_scale=(0.5, 5.0))
surprise_data = Dataset.load_from_df(df_ratings_sample, reader)
# surprise_data = Dataset.load_from_df(df_ratings, reader)

# Train and Evaluate Model

Using default parameters, establish syntax for fitting, evaluating, and saving a model.

Hyperparameter tuning will be done via sweep job in AzureML on the following:

* `n_factors` 
* `lr_all` 
* `reg_all` 

In [2]:
train_set, test_set = train_test_split(surprise_data, test_size=0.2, random_state=seed)

svd_model = SVD(
  n_factors=20, # default
  n_epochs=20, # default
  random_state=seed
)

svd_model.fit(train_set)

predictions = svd_model.test(test_set)
# RMSE as the primary metric to evaluate (minimize)
rmse = accuracy.rmse(predictions)
print(rmse) 
mae = accuracy.mae(predictions)
print(mae)

RMSE: 0.8522
0.8521971857568799
MAE:  0.6461
0.6461339414193713


5-fold cross-validation

In [3]:
cross_validate(svd_model, surprise_data, measures=["RMSE", "MAE"], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8503  0.8508  0.8519  0.8522  0.8510  0.8513  0.0007  
MAE (testset)     0.6446  0.6453  0.6465  0.6462  0.6456  0.6456  0.0006  
Fit time          43.02   44.23   44.89   44.13   43.88   44.03   0.61    
Test time         10.44   8.80    10.36   8.76    10.27   9.73    0.77    


{'test_rmse': array([0.8503484 , 0.85083543, 0.85187028, 0.85218144, 0.85101775]),
 'test_mae': array([0.6446485 , 0.64533745, 0.64650627, 0.64615009, 0.64558509]),
 'fit_time': (43.01675081253052,
  44.23181891441345,
  44.89074158668518,
  44.12594175338745,
  43.88174223899841),
 'test_time': (10.43651795387268,
  8.80109977722168,
  10.355739831924438,
  8.764165163040161,
  10.269036769866943)}

In [ ]:
# save model artifact
# import pickle

# with open('svd_model.pkl', 'wb') as file:
#   pickle.dump(svd_model, file)

# Generate Recommendations via SVD

The model can be used to output predicted ratings for a given user on films they have not yet rated.

In [5]:
def recommend_for_user_svd(df_ratings, df_films, user_id, top_n=10):
  """
  Recommend movies for a given user using the trained SVD model.

  Args:
    df_ratings: A dataframe of film ratings with columns: user_id, movie_id, rating
    df_films: A dimension table dataframe of films that provides the title and genre(s) for a given movie_id
    user_id: A specific user_id present in df_ratings
    top_n: # of recommendations to return

  Return: A dataframe with top_n rows of recommended titles, genres, and predicted ratings
  """
  all_movie_ids = df_ratings['movie_id'].unique()

  if user_id not in df_ratings['user_id'].unique():
    print("This user is not present in the ratings dataset provided")
    return None

  # Movies the user has already rated
  rated = df_ratings[df_ratings['user_id'] == user_id]['movie_id'].unique()
  # Candidate movies = all movies in sample that user has NOT rated
  candidate_movie_ids = [mid for mid in all_movie_ids if mid not in rated]

  # use model to predict ratings for not-yet-rated movie_id's
  preds = []
  for mid in candidate_movie_ids:
      pred = svd_model.predict(user_id, mid)
      preds.append((mid, pred.est))

  # Sort by predicted rating (high to low)
  preds_sorted = sorted(preds, key=lambda x: x[1], reverse=True)[:top_n]
  top_movie_ids = [mid for (mid, _) in preds_sorted]
  top_scores = [score for (_, score) in preds_sorted]

  # Map movie_id to title + genre
  recs = df_films[df_films['movie_id'].isin(top_movie_ids)][['movie_id', 'title', 'genres']]
  # Preserve order of top_movie_ids
  recs = recs.set_index('movie_id').loc[top_movie_ids].reset_index()
  recs['predicted_rating'] = top_scores

  return recs


In [ ]:
df_films = pd.read_csv(os.path.join(data_dir, 'movies.csv')).rename(columns={'movieId':'movie_id'})

recommend_for_user_svd(df_ratings, df_films, user_id=999999, top_n=10)

,movie_id,title,genres,predicted_rating
0,170705,Band of Brothers (2001),Action|Drama|War,4.114967
1,1203,12 Angry Men (1957),Drama,4.039348
2,1204,Lawrence of Arabia (1962),Adventure|Drama|War,4.026055
3,171011,Planet Earth II (2016),Documentary,4.026030
4,169252,Everything Will Be OK (2006),Animation|Drama,4.008492
5,159817,Planet Earth (2006),Documentary,4.006651
6,7396,Scenes From a Marriage (Scener ur ett äktenska...,Drama,4.003602
7,198185,Twin Peaks (1989),Drama|Mystery,4.000201
8,32649,"Special Day, A (Giornata particolare, Una) (1977)",Drama|War,3.991357
9,26587,"Decalogue, The (Dekalog) (1989)",Crime|Drama|Romance,3.990987
